In [1]:
# set path
%cd /work/project/escience/ruttho/FCC-ee_SimpleDelphesAnalysis/Analysis/

/work/project/escience/ruttho/FCC-ee_SimpleDelphesAnalysis/Analysis


In [7]:
#!/usr/bin/env python3
import uproot
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import awkward as ak
import seaborn as sns
import os

# Configuration for testing vs production
TESTING_MODE = True  # Set to False for full analysis
MAX_EVENTS = 1 if TESTING_MODE else None  # None means read all events
print(f"Running in {'TESTING' if TESTING_MODE else 'PRODUCTION'} mode")
# Load the ROOT files and access the trees
HZ4Lep = uproot.open("Bg1_hz.root")
ZWW4Lep = uproot.open("Bg2_zww.root")
HZ4LepLFV = uproot.open("Sn1_hzLFV.root")
HZ4Lep_tree = HZ4Lep["Delphes"]
ZWW4Lep_tree = ZWW4Lep["Delphes"]
HZ4LepLFV_tree = HZ4LepLFV["Delphes"]

# Check for events with exactly 4 leptons
column_arrays = ["MissingET_size","MissingET.Eta","MissingET.Phi","MissingET.MET",
                 "Jet_size","Jet.Phi","Jet.Eta","Jet.PT","Jet.Mass",
                 "Electron_size","Electron.Eta","Electron.Phi","Electron.PT",
                 "Muon_size","Muon.Eta","Muon.Phi","Muon.PT"]
HZ4Lep_array = HZ4Lep_tree.arrays(column_arrays,  entry_stop=MAX_EVENTS)
ZWW4Lep_array = ZWW4Lep_tree.arrays(column_arrays,  entry_stop=MAX_EVENTS)
HZ4LepLFV_array = HZ4LepLFV_tree.arrays(column_arrays, entry_stop=MAX_EVENTS)

Running in TESTING mode


In [12]:
HZ4Lep_tree.keys()

['Event',
 'Event/Event.fUniqueID',
 'Event/Event.fBits',
 'Event/Event.Number',
 'Event/Event.ReadTime',
 'Event/Event.ProcTime',
 'Event/Event.ProcessID',
 'Event/Event.MPI',
 'Event/Event.Weight',
 'Event/Event.CrossSection',
 'Event/Event.CrossSectionError',
 'Event/Event.Scale',
 'Event/Event.AlphaQED',
 'Event/Event.AlphaQCD',
 'Event/Event.ID1',
 'Event/Event.ID2',
 'Event/Event.X1',
 'Event/Event.X2',
 'Event/Event.ScalePDF',
 'Event/Event.PDF1',
 'Event/Event.PDF2',
 'Event_size',
 'Weight',
 'Weight/Weight.fUniqueID',
 'Weight/Weight.fBits',
 'Weight/Weight.Weight',
 'Weight_size',
 'Particle',
 'Particle/Particle.fUniqueID',
 'Particle/Particle.fBits',
 'Particle/Particle.PID',
 'Particle/Particle.Status',
 'Particle/Particle.IsPU',
 'Particle/Particle.M1',
 'Particle/Particle.M2',
 'Particle/Particle.D1',
 'Particle/Particle.D2',
 'Particle/Particle.Charge',
 'Particle/Particle.Mass',
 'Particle/Particle.E',
 'Particle/Particle.Px',
 'Particle/Particle.Py',
 'Particle/Parti

In [ ]:
qw.keys()

In [3]:
def Plot_ThreeData(arr1,arr2,arr3,setting,other_config='',exception=False):
    fig=plt.figure()
    ax=fig.add_subplot(111)
    if not exception:
        n_arr=[ak.flatten(arr3[setting["name"]]), ak.flatten(arr2[setting["name"]]), ak.flatten(arr1[setting["name"]])]
    else:
        n_arr=[arr3, arr2, arr1]
    sns.set_theme(style="ticks")
    sns.histplot(n_arr, bins=setting["bins"], binrange=setting["binrange"], ax=ax, 
                element=setting["element"], alpha=setting["alpha"],palette=setting['colors'][::-1], 
                stat=setting["stat"],common_norm=False,
                multiple=setting["multiple"]
                )
    ax.set_title(setting['name']+" - "+setting['prefix'])
    ax.set_xlabel(setting['xlabel'])
    ax.set_ylabel("Density")
    ax.legend(labels=setting['labels'])
    if setting['log_scale']:
        ax.set_yscale("log")
    png_name = f"{setting['prefix']}_{setting['name']}{other_config}.png"
    plt.savefig(f"figure/test/{png_name}")
    print(f"-> {png_name} has been ploted!")
    # plt.show()  
    plt.close()
    return 1

def Plot_ECDF(arr1,arr2,arr3,setting,other_config=''):
    fig=plt.figure()
    ax=fig.add_subplot(111)
    n_arr=[ak.flatten(arr3[setting["name"]]), ak.flatten(arr2[setting["name"]]), ak.flatten(arr1[setting["name"]])]
    sns.set_theme(style="ticks")
    sns.ecdfplot(data=n_arr, ax=ax, palette=setting['colors'][::-1])
    ax.set_title(setting['name']+" - "+setting['prefix'])
    ax.set_xlabel(setting['xlabel'])
    ax.set_ylabel("Density")
    ax.legend(labels=setting['labels'])
    if setting['log_scale']:
        ax.set_yscale("log")
    png_name = f"edcf_{setting['prefix']}_{setting['name']}{other_config}.png"
    plt.savefig(f"figure/test/{png_name}")
    print(f"-> {png_name} has been ploted!")
    # plt.show()  
    plt.close()
    return 1

def Plot_singleData(arr,setting,other_config='',samplename=""):
    fig=plt.figure()
    ax=fig.add_subplot(111)
    n_arr=ak.flatten(arr[setting["name"]])
    sns.set_theme(style="ticks")
    sns.histplot(n_arr, bins=setting["bins"], binrange=setting["binrange"], ax=ax, 
                element=setting["element"], alpha=setting["alpha"],palette=setting['colors'], 
                stat=setting["stat"],
                multiple=setting["multiple"]
                )
    ax.set_title(setting['name']+" - "+setting['prefix'])
    ax.set_xlabel(setting['xlabel'])
    ax.set_ylabel("Density")
    if setting['log_scale']:
        ax.set_yscale("log")
    png_name = f"{samplename}{setting['prefix']}_{setting['name']}{other_config}.png"
    plt.savefig(f"figure/test/{png_name}")
    print(f"-> {png_name} has been ploted!")
    # plt.show()  
    plt.close()
    return 1

def firstcut_4lFinalState(array,name):
    mask = (array["Electron_size"] + array["Muon_size"] == 4)
    print(f"    {name}:{np.sum(mask)}/{len(mask)} : {np.sum(mask)/len(mask)*100:.2f}%")
    return mask

def secondcut_EandMuNotEven(array,name,pmask):
    mask = (array["Electron_size"]%2!=0) & (array["Muon_size"]%2!=0) & pmask
    print(f"    {name}:{np.sum(mask)}/{len(mask)} : {np.sum(mask)/len(mask)*100:.2f}%")
    return mask


In [4]:
# Abandoned cut
def thirdcut_MET(array,name,pmask,threshold=7):
    mask = (ak.firsts(array["MissingET.MET"])<threshold) & pmask
    print(f"    {name}:{np.sum(mask)}/{len(mask)} : {np.sum(mask)/len(mask)*100:.2f}%")
    return mask

def check_drFromMET(HZ4LepLFV_array,ZWW4Lep_array,HZ4Lep_array,config):
    # Calculate delta R between MET and each lepton
    array=[HZ4LepLFV_array,ZWW4Lep_array,HZ4Lep_array]
    dR=[]
    deta=[]
    dphi=[]
    for i in range(3):
        dphi_electron_MET = ak.flatten(ak.firsts(array[i]["MissingET.Phi"]) - array[i]["Electron.Phi"])
        dphi_electron_MET_Norm = (dphi_electron_MET + np.pi) % (2 * np.pi) - np.pi
        dphi_muon_MET = ak.flatten(ak.firsts(array[i]["MissingET.Phi"]) - array[i]["Muon.Phi"])
        dphi_muon_MET_Norm = (dphi_muon_MET + np.pi) % (2 * np.pi) - np.pi
        deta_electron_MET = ak.flatten(ak.firsts(array[i]["MissingET.Eta"]) - array[i]["Electron.Eta"])
        deta_muon_MET = ak.flatten(ak.firsts(array[i]["MissingET.Eta"]) - array[i]["Muon.Eta"])
        dR_electron = np.sqrt(deta_electron_MET**2 + dphi_electron_MET_Norm**2)
        dR_muon = np.sqrt(deta_muon_MET**2 + dphi_muon_MET_Norm**2)
        dR_all = ak.concatenate([dR_electron,dR_muon])
        dR.append(dR_all)
        deta.append(ak.concatenate([deta_electron_MET,deta_muon_MET]))
        dphi.append(ak.concatenate([dphi_electron_MET_Norm,dphi_muon_MET_Norm]))
    
    config['xlabel']=''
    config['log_scale']=False
    config['name']='dRFromMETtoLepton'
    config["binrange"] = (0,10)
    config['bins']=100
    Plot_ThreeData(dR[0],dR[1],dR[2],config,exception=True)
    config['name']='detaFromMETtoLepton'
    config["binrange"] = (-10,10)
    config['bins']=200
    Plot_ThreeData(deta[0],deta[1],deta[2],config,exception=True)
    config['name']='dphiFromMETtoLepton'
    config["binrange"] = (-np.pi,np.pi)
    config['bins']=100
    Plot_ThreeData(dphi[0],dphi[1],dphi[2],config,exception=True)
    config['name']='Jet_size'
    config["binrange"] = (-0.5,9.5)
    config['bins']=10
    Plot_ThreeData(HZ4LepLFV_array['Jet_size'],ZWW4Lep_array['Jet_size'],HZ4Lep_array['Jet_size'],config,exception=True)

def dphi_MET(array,name):
    dphi = ak.flatten(ak.firsts(array["MissingET.Phi"]) - array[name])
    dphi = ak.where(dphi > np.pi, 2*np.pi - dphi, dphi)
    return dphi

def deta_MET(array,name):
    deta = ak.flatten(ak.firsts(array["MissingET.Eta"]) - array[name])
    return deta

In [ ]:
def Data_visualize(HZ4LepLFV_array,ZWW4Lep_array,HZ4Lep_array,prefix,additional_plot=False):
    pc={'prefix':prefix,'name':'Electron.PT','bins':65,'binrange':(0,130),'element':"step",'alpha':0.4,
                 'xlabel':"GeV",'colors':["#FF9966","#6699cc","#66cc99"],'labels':["HZ4LepLFV","ZWW4Lep","HZ4Lep"],
                 'multiple':"layer",'stat':"density",'log_scale':False}
    Plot_ThreeData(HZ4LepLFV_array,ZWW4Lep_array,HZ4Lep_array,pc)
    pc['name']='Jet.PT'
    Plot_ThreeData(HZ4LepLFV_array,ZWW4Lep_array,HZ4Lep_array,pc)
    pc['name']='Jet.Mass'
    pc['log_scale']=True
    Plot_ThreeData(HZ4LepLFV_array,ZWW4Lep_array,HZ4Lep_array,pc)
    pc['log_scale']=False
    pc['name']="Muon.PT"
    Plot_ThreeData(HZ4LepLFV_array,ZWW4Lep_array,HZ4Lep_array,pc)
    pc['xlabel']=''
    pc['name']="Electron.Eta"
    pc['binrange']=(-10,10)
    pc['bins']=200
    Plot_ThreeData(HZ4LepLFV_array,ZWW4Lep_array,HZ4Lep_array,pc)
    pc['name']="Muon.Eta"
    Plot_ThreeData(HZ4LepLFV_array,ZWW4Lep_array,HZ4Lep_array,pc)
    pc['name']="Electron.Phi"
    pc['binrange']=(-2*np.pi,2*np.pi)
    pc['bins']=140
    Plot_ThreeData(HZ4LepLFV_array,ZWW4Lep_array,HZ4Lep_array,pc)
    pc['name']="Muon.Phi"
    Plot_ThreeData(HZ4LepLFV_array,ZWW4Lep_array,HZ4Lep_array,pc)
    pc['xlabel']='GeV'
    pc["name"]="MissingET.MET"
    pc['binrange']=(0,130)
    pc['bins']=130
    pc['log_scale']=True
    Plot_ThreeData(HZ4LepLFV_array,ZWW4Lep_array,HZ4Lep_array,pc)
    pc['binrange']=(0,20)
    pc['bins']=200
    Plot_ThreeData(HZ4LepLFV_array,ZWW4Lep_array,HZ4Lep_array,pc,other_config="_20GeVCut")
    pc['binrange']=(0,7.5)
    pc['bins']=75
    Plot_ThreeData(HZ4LepLFV_array,ZWW4Lep_array,HZ4Lep_array,pc,other_config="_7.5GeVCut")



In [ ]:
print("Plot uncut events")
Data_visualize(HZ4LepLFV_array,ZWW4Lep_array,HZ4Lep_array,"0Uncut")
print("Applying first cut: Events with exactly 4 leptons...")
mask_dict={'1st':{}}
mask_dict['1st']["HZ4LepLFV"] = firstcut_4lFinalState(HZ4LepLFV_array,"HZ4LepLFV")
mask_dict['1st']["ZWW4Lep"] = firstcut_4lFinalState(ZWW4Lep_array,"ZWW4Lep")
mask_dict['1st']["HZ4Lep"] = firstcut_4lFinalState(HZ4Lep_array,"HZ4Lep")
Data_visualize(HZ4LepLFV_array[mask_dict['1st']["HZ4LepLFV"]],ZWW4Lep_array[mask_dict['1st']["ZWW4Lep"]],
               HZ4Lep_array[mask_dict['1st']["HZ4Lep"]],"1stCut")
print("Applying second cut: Events with at least one electron and one muon...")
mask_dict['2nd'] = {}
mask_dict['2nd']["HZ4LepLFV"] = secondcut_EandMuNotEven(HZ4LepLFV_array,"HZ4LepLFV",mask_dict['1st']["HZ4LepLFV"])
mask_dict['2nd']["ZWW4Lep"] = secondcut_EandMuNotEven(ZWW4Lep_array,"ZWW4Lep",mask_dict['1st']["ZWW4Lep"])
mask_dict['2nd']["HZ4Lep"] = secondcut_EandMuNotEven(HZ4Lep_array,"HZ4Lep",mask_dict['1st']["HZ4Lep"])
Data_visualize(HZ4LepLFV_array[mask_dict['2nd']["HZ4LepLFV"]],ZWW4Lep_array[mask_dict['2nd']["ZWW4Lep"]],
               HZ4Lep_array[mask_dict['2nd']["HZ4Lep"]],"2ndCut",additional_plot=True)



Plot uncut events
-> 0Uncut_Electron.PT.png has been ploted!
-> 0Uncut_Jet.PT.png has been ploted!
-> 0Uncut_Jet.Mass.png has been ploted!
-> 0Uncut_Muon.PT.png has been ploted!
-> 0Uncut_Electron.Eta.png has been ploted!
-> 0Uncut_Muon.Eta.png has been ploted!
-> 0Uncut_Electron.Phi.png has been ploted!
-> 0Uncut_Muon.Phi.png has been ploted!
-> 0Uncut_MissingET.MET.png has been ploted!
-> 0Uncut_MissingET.MET_20GeVCut.png has been ploted!
-> 0Uncut_MissingET.MET_7.5GeVCut.png has been ploted!
Applying first cut: Events with exactly 4 leptons...
    HZ4LepLFV:6605/10000 : 66.05%
    ZWW4Lep:5893/10000 : 58.93%
    HZ4Lep:5867/10000 : 58.67%
-> 1stCut_Electron.PT.png has been ploted!
-> 1stCut_Jet.PT.png has been ploted!
-> 1stCut_Jet.Mass.png has been ploted!
-> 1stCut_Muon.PT.png has been ploted!
-> 1stCut_Electron.Eta.png has been ploted!
-> 1stCut_Muon.Eta.png has been ploted!
-> 1stCut_Electron.Phi.png has been ploted!
-> 1stCut_Muon.Phi.png has been ploted!
-> 1stCut_MissingET.ME

1